In [3]:
import json
from datasets import Dataset, DatasetDict
import pandas as pd
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, DataCollatorWithPadding
import torch

# 加载自定义格式的JSON文件
with open('custom_conversations.json', 'r') as f:
    data = json.load(f)

# 自定义数据处理函数
def process_conversations(data):
    conversations = []
    for conv in data:
        for turn in conv['chat']:
            conversations.append({
                "conversation_id": conv["id"],
                "turn_id": turn["id"],
                "user": turn["user_msg"],
                "assistant": turn["bot_response"]
            })
    return conversations



# 将模型移动到GPU（如果可用）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 应用数据处理函数
processed_data = process_conversations(data)

# 创建Dataset对象
dataset = Dataset.from_pandas(pd.DataFrame(processed_data))

# 选择预训练模型的分词器
tokenizer = AutoTokenizer.from_pretrained("shenzhi-wang/Llama3-8B-Chinese-Chat")

# 定义分词函数
def tokenize_function(examples):
    return tokenizer(examples['user'], examples['assistant'], padding="max_length", truncation=True, max_length=512)

# 对数据集进行分词和编码
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 将数据集拆分为训练集和验证集
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# 使用DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer)

# 选择预训练模型
model = AutoModelForCausalLM.from_pretrained("shenzhi-wang/Llama3-8B-Chinese-Chat").half().to(device)

# # 设置训练参数
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     save_total_limit=1,
# )

# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=True,  # 启用半精度训练
    dataloader_num_workers=4,
    deepspeed="ds_config.json",  # 添加DeepSpeed配置文件进行分布式训练
)

# 定义Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator
)

# 开始训练
trainer.train()

# 保存模型
model.save_pretrained("./fine_tuned_llama3")
tokenizer.save_pretrained("./fine_tuned_llama3")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:  25%|██▌       | 1/4 [00:01<00:05,  1.85s/it]

In [ ]:
{
  "train_batch_size": 16,
  "gradient_accumulation_steps": 4,
  "fp16": {
    "enabled": true
  },
  "zero_optimization": {
    "stage": 2,
    "allgather_partitions": true,
    "allgather_bucket_size": 2e8,
    "overlap_comm": true,
    "reduce_scatter": true,
    "reduce_bucket_size": 2e8,
    "contiguous_gradients": true
  },
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 2e-5,
      "betas": [0.8, 0.999],
      "eps": 1e-8,
      "weight_decay": 3e-7
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 2e-5,
      "warmup_num_steps": 1000
    }
  }
}


In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("shenzhi-wang/Llama3-8B-Chinese-Chat") 
model = AutoModelForCausalLM.from_pretrained("shenzhi-wang/Llama3-8B-Chinese-Chat")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/anonymous/anaconda3/envs/nursingllm/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
/home/anonymous/anaconda3/envs/nursingllm/lib/python3.10/site-packages/torch/cuda/__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.05s/it]


In [9]:
import json
from datasets import Dataset, DatasetDict

# 加载自定义格式的JSON文件
with open('custom_conversations.json', 'r') as f:
    data = json.load(f)

In [11]:
import pandas as pd

def process_conversations(data):
    conversations = []
    for conv in data:
        for turn in conv['chat']:
            conversations.append({
                "conversation_id": conv["id"],
                "turn_id": turn["id"],
                "user": turn["user_msg"],
                "assistant": turn["bot_response"]
            })
    return conversations

processed_data = process_conversations(data)

# 创建Dataset对象
dataset = Dataset.from_pandas(pd.DataFrame(processed_data))



In [13]:
from transformers import AutoTokenizer

# # 选择预训练模型的分词器
# tokenizer = AutoTokenizer.from_pretrained("shenzhi-wang/Llama3-8B-Chinese-Chat")

# 定义分词函数
def tokenize_function(examples):
    return tokenizer(examples['user'], examples['assistant'], padding="max_length", truncation=True)

# 对数据集进行分词
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map:   0%|          | 0/4 [00:00<?, ? examples/s]Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Map: 100%|██████████| 4/4 [00:00<00:00, 253.94 examples/s]


In [14]:
# 将数据集拆分为训练集和验证集
split_dataset = tokenized_datasets.train_test_split(test_size=0.2)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']


In [16]:
from transformers import TrainingArguments, Trainer, AutoModelForCausalLM


# 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
)

# 定义 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# 开始训练
trainer.train()


/home/anonymous/anaconda3/envs/nursingllm/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]

ValueError: expected sequence of length 18 at dim 1 (got 19)

/home/anonymous/anaconda3/envs/nursingllm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:07<00:00,  1.97s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 

In [3]:
device

device(type='cpu')

In [7]:
model

NameError: name 'model' is not defined

In [2]:

from datasets import load_dataset

# # 加载C4数据集
# c4 = load_dataset('c4', 'en')

# # 加载BioASQ数据集
# bioasq = load_dataset('bioasq')

# # 加载ECtHR数据集
# ecthr = load_dataset('ecthr_cases')

# 预处理示例：BioASQ数据集
def preprocess_bioasq(examples):
    inputs = [question for question in examples['question']]
    return tokenizer(inputs, max_length=512, truncation=True, padding='max_length')

# tokenized_bioasq = bioasq.map(preprocess_bioasq, batched=True)

# # 示例：打印一些数据
# print(c4['train'][0])
# print(bioasq['train'][0])
# print(ecthr['train'][0])

dataset_loaded = load_dataset('boolq')

In [5]:
daily_dialog = load_dataset('daily_dialog')
print(daily_dialog['train'][0])


Generating test split: 100%|██████████| 1000/1000 [00:00<00:00, 27295.83 examples/s]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [6]:
daily_dialog['train'][0]['dialog']

['Say , Jim , how about going for a few beers after dinner ? ',
 ' You know that is tempting but is really not good for our fitness . ',
 ' What do you mean ? It will help us to relax . ',
 " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
 " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
 ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
 " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
 ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
 " Good.Let ' s go now . ",
 ' All right . ']

In [3]:
model.chat("你好", max_length=100)


AttributeError: 'LlamaForCausalLM' object has no attribute 'chat'